In [8]:
import numpy as np
import pandas as pd
import os
import itertools
import torch
import torch.nn as nn
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.data import Dataset
from torch_geometric.loader import dataloader
from sklearn.model_selection import train_test_split
from torch_geometric_temporal.signal import temporal_signal_split
from torch_geometric_temporal.nn.recurrent import GConvLSTM, GConvGRU
from torch_geometric_temporal.signal import StaticGraphTemporalSignal

In [9]:
data_df = pd.read_csv("datasetsrip - synthetic-prcp-SM-flood-data-v1.csv.csv")
data_df.drop(['Unnamed: 0'], axis=1, inplace=True)
data_df

,Prcp1,SM1,Prcp2,SM2,Prcp3,SM3,flood1,flood2,flood3
0,1.539606,0.590441,2.511432,0.474052,4.585283,0.484656,0,0,0
1,2.274125,1.249220,7.461011,2.159259,17.690303,3.474562,1,1,1
2,1.000114,0.106579,1.842560,0.010000,3.395878,0.010000,0,0,0
3,1.360013,0.429366,2.085287,0.212607,3.610686,0.137334,0,0,0
4,1.158710,0.248820,1.282190,0.010000,1.592812,0.010000,0,0,0
...,...,...,...,...,...,...,...,...,...
360,1.357030,0.426691,2.020841,0.197245,3.431358,0.103488,0,0,0
361,1.154324,0.244887,3.097722,0.201381,6.761527,0.473908,0,0,1
362,3.315756,2.183443,2.927501,2.330118,3.422077,2.060617,1,1,0
363,1.779925,0.805979,2.532323,0.718383,4.327215,0.680561,0,0,0


GConvGRU

In [12]:
def test_1layer_GConvGRU(df):
    x = np.zeros([df.shape[0], 3, 2])
    x[:, 0, 0] = df['Prcp1'].values
    x[:, 0, 1] = df['SM1'].values
    x[:, 1, 0] = df['Prcp2'].values
    x[:, 1, 1] = df['SM2'].values
    x[:, 2, 0] = df['Prcp3'].values
    x[:, 2, 1] = df['SM3'].values

    y = np.zeros([df.shape[0], 3, 1])
    y[:, 0, 0] = df['flood1'].values
    y[:, 1, 0] = df['flood2'].values
    y[:, 2, 0] = df['flood3'].values

    dataset = StaticGraphTemporalSignal(
        edge_index = np.array([[0,2], [1,1]], dtype=np.compat.long),
        edge_weight = np.array([1.0,1.0], dtype=float),
        features = x,
        targets = y
    )

    train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.75)

    class TemporalGNN(torch.nn.Module):
        def __init__(self, dim_in):
            super().__init__()
            self.recurrent = GConvGRU(dim_in, 16, 1)
            self.linear = Linear(16, 1)

        def forward(self, x, edge_index, edge_weight):
            x = self.recurrent(x, edge_index, edge_weight).relu()
            x = self.linear(x)
            return torch.sigmoid(x)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
 
    print(device)
    model = TemporalGNN(dim_in=2).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.BCELoss().to(device)

    model.train()
    for epoch in range(10):
        cost = 0
        for time, snapshot in enumerate(train_dataset):
            snapshot.to(device)
            y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
            cost = criterion(y_hat, snapshot.y)
            cost.backward()
            optimizer.step()
            optimizer.zero_grad()
        print('Epoch: {:03d}, Loss: {:.5f}'.format(epoch, cost.item()))

    model.eval()
    y_pred = []
    y_true = []
    for time, snapshot in enumerate(test_dataset):
        snapshot.to(device)
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
        y_pred.append(y_hat.detach().cpu().numpy())
        y_true.append(snapshot.y.detach().cpu().numpy())
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    print('Test Accuracy: {:.4f}'.format((y_pred.round() == y_true).mean()))

    del model
    torch.cuda.empty_cache()

test_1layer_GConvGRU(data_df)

cpu
Epoch: 000, Loss: 0.38113
Epoch: 001, Loss: 0.25151
Epoch: 002, Loss: 0.20746
Epoch: 003, Loss: 0.18814
Epoch: 004, Loss: 0.17641
Epoch: 005, Loss: 0.16776
Epoch: 006, Loss: 0.16098
Epoch: 007, Loss: 0.15548
Epoch: 008, Loss: 0.15102
Epoch: 009, Loss: 0.14720
Test Accuracy: 0.9239


In [20]:
def test_1layer_GConvGRU(df):
    x = np.zeros([df.shape[0], 3, 2])
    x[:, 0, 0] = df['Prcp1'].values
    x[:, 0, 1] = df['SM1'].values
    x[:, 1, 0] = df['Prcp2'].values
    x[:, 1, 1] = df['SM2'].values
    x[:, 2, 0] = df['Prcp3'].values
    x[:, 2, 1] = df['SM3'].values

    y = np.zeros([df.shape[0], 3, 1])
    y[:, 0, 0] = df['flood1'].values
    y[:, 1, 0] = df['flood2'].values
    y[:, 2, 0] = df['flood3'].values

    dataset = StaticGraphTemporalSignal(
        edge_index = np.array([[0,2], [1,1]], dtype=np.compat.long),
        edge_weight = np.array([1.0,1.0], dtype=float),
        features = x,
        targets = y
    )

    train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.75)

    class TemporalGNN(torch.nn.Module):
        def __init__(self, dim_in):
            super().__init__()
            self.recurrent = GConvGRU(dim_in, 16, 1)
            self.linear = Linear(16, 1)

        def forward(self, x, edge_index, edge_weight):
            x = self.recurrent(x, edge_index, edge_weight).relu()
            x = self.linear(x)
            return torch.sigmoid(x)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
 
    print(device)
    model = TemporalGNN(dim_in=2).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.BCELoss().to(device)

    model.train()
    for epoch in range(20):
        cost = 0
        for time, snapshot in enumerate(train_dataset):
            snapshot.to(device)
            y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
            cost = criterion(y_hat, snapshot.y)
            cost.backward()
            optimizer.step()
            optimizer.zero_grad()
        print('Epoch: {:03d}, Loss: {:.5f}'.format(epoch, cost.item()))

    model.eval()
    y_pred = []
    y_true = []
    for time, snapshot in enumerate(test_dataset):
        snapshot.to(device)
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
        y_pred.append(y_hat.detach().cpu().numpy())
        y_true.append(snapshot.y.detach().cpu().numpy())
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    print('Test Accuracy: {:.4f}'.format((y_pred.round() == y_true).mean()))

    del model
    torch.cuda.empty_cache()

test_1layer_GConvGRU(data_df)

cpu
Epoch: 000, Loss: 0.38139
Epoch: 001, Loss: 0.26604
Epoch: 002, Loss: 0.23100
Epoch: 003, Loss: 0.21486
Epoch: 004, Loss: 0.20595
Epoch: 005, Loss: 0.20018
Epoch: 006, Loss: 0.19630
Epoch: 007, Loss: 0.19520
Epoch: 008, Loss: 0.19336
Epoch: 009, Loss: 0.19135
Epoch: 010, Loss: 0.18936
Epoch: 011, Loss: 0.18495
Epoch: 012, Loss: 0.18197
Epoch: 013, Loss: 0.17803
Epoch: 014, Loss: 0.17585
Epoch: 015, Loss: 0.17372
Epoch: 016, Loss: 0.17672
Epoch: 017, Loss: 0.17499
Epoch: 018, Loss: 0.17343
Epoch: 019, Loss: 0.17322
Test Accuracy: 0.9203


In [14]:
def test_1layer_GConvGRU(df):
    x = np.zeros([df.shape[0], 3, 2])
    x[:, 0, 0] = df['Prcp1'].values
    x[:, 0, 1] = df['SM1'].values
    x[:, 1, 0] = df['Prcp2'].values
    x[:, 1, 1] = df['SM2'].values
    x[:, 2, 0] = df['Prcp3'].values
    x[:, 2, 1] = df['SM3'].values

    y = np.zeros([df.shape[0], 3, 1])
    y[:, 0, 0] = df['flood1'].values
    y[:, 1, 0] = df['flood2'].values
    y[:, 2, 0] = df['flood3'].values

    dataset = StaticGraphTemporalSignal(
        edge_index = np.array([[0,2], [1,1]], dtype=np.compat.long),
        edge_weight = np.array([1.0,1.0], dtype=float),
        features = x,
        targets = y
    )

    train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.75)

    class TemporalGNN(torch.nn.Module):
        def __init__(self, dim_in):
            super().__init__()
            self.recurrent = GConvGRU(dim_in, 16, 1)
            self.linear = Linear(16, 1)

        def forward(self, x, edge_index, edge_weight):
            x = self.recurrent(x, edge_index, edge_weight).relu()
            x = self.linear(x)
            return torch.sigmoid(x)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
 
    print(device)
    model = TemporalGNN(dim_in=2).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.BCELoss().to(device)

    model.train()
    for epoch in range(50):
        cost = 0
        for time, snapshot in enumerate(train_dataset):
            snapshot.to(device)
            y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
            cost = criterion(y_hat, snapshot.y)
            cost.backward()
            optimizer.step()
            optimizer.zero_grad()
        print('Epoch: {:03d}, Loss: {:.5f}'.format(epoch, cost.item()))

    model.eval()
    y_pred = []
    y_true = []
    for time, snapshot in enumerate(test_dataset):
        snapshot.to(device)
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
        y_pred.append(y_hat.detach().cpu().numpy())
        y_true.append(snapshot.y.detach().cpu().numpy())
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    print('Test Accuracy: {:.4f}'.format((y_pred.round() == y_true).mean()))

    del model
    torch.cuda.empty_cache()

test_1layer_GConvGRU(data_df)

cpu
Epoch: 000, Loss: 0.37592
Epoch: 001, Loss: 0.27273
Epoch: 002, Loss: 0.23945
Epoch: 003, Loss: 0.22304
Epoch: 004, Loss: 0.21306
Epoch: 005, Loss: 0.20626
Epoch: 006, Loss: 0.20058
Epoch: 007, Loss: 0.19658
Epoch: 008, Loss: 0.19345
Epoch: 009, Loss: 0.19061
Epoch: 010, Loss: 0.18528
Epoch: 011, Loss: 0.18269
Epoch: 012, Loss: 0.18025
Epoch: 013, Loss: 0.17555
Epoch: 014, Loss: 0.17831
Epoch: 015, Loss: 0.17494
Epoch: 016, Loss: 0.17590
Epoch: 017, Loss: 0.17199
Epoch: 018, Loss: 0.16869
Epoch: 019, Loss: 0.17081
Epoch: 020, Loss: 0.16592
Epoch: 021, Loss: 0.16627
Epoch: 022, Loss: 0.16374
Epoch: 023, Loss: 0.16316
Epoch: 024, Loss: 0.16355
Epoch: 025, Loss: 0.16126
Epoch: 026, Loss: 0.16031
Epoch: 027, Loss: 0.15885
Epoch: 028, Loss: 0.15768
Epoch: 029, Loss: 0.15589
Epoch: 030, Loss: 0.15430
Epoch: 031, Loss: 0.15224
Epoch: 032, Loss: 0.15048
Epoch: 033, Loss: 0.14830
Epoch: 034, Loss: 0.14615
Epoch: 035, Loss: 0.14385
Epoch: 036, Loss: 0.14123
Epoch: 037, Loss: 0.13907
Epoch: 0

In [22]:
def test_1layer_GConvGRU(df):
    x = np.zeros([df.shape[0], 3, 2])
    x[:, 0, 0] = df['Prcp1'].values
    x[:, 0, 1] = df['SM1'].values
    x[:, 1, 0] = df['Prcp2'].values
    x[:, 1, 1] = df['SM2'].values
    x[:, 2, 0] = df['Prcp3'].values
    x[:, 2, 1] = df['SM3'].values

    y = np.zeros([df.shape[0], 3, 1])
    y[:, 0, 0] = df['flood1'].values
    y[:, 1, 0] = df['flood2'].values
    y[:, 2, 0] = df['flood3'].values

    dataset = StaticGraphTemporalSignal(
        edge_index = np.array([[0,2], [1,1]], dtype=np.compat.long),
        edge_weight = np.array([1.0,1.0], dtype=float),
        features = x,
        targets = y
    )

    train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.75)

    class TemporalGNN(torch.nn.Module):
        def __init__(self, dim_in):
            super().__init__()
            self.recurrent = GConvGRU(dim_in, 16, 1)
            self.linear = Linear(16, 1)

        def forward(self, x, edge_index, edge_weight):
            x = self.recurrent(x, edge_index, edge_weight).relu()
            x = self.linear(x)
            return torch.sigmoid(x)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
 
    print(device)
    model = TemporalGNN(dim_in=2).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.BCELoss().to(device)

    model.train()
    for epoch in range(100):
        cost = 0
        for time, snapshot in enumerate(train_dataset):
            snapshot.to(device)
            y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
            cost = criterion(y_hat, snapshot.y)
            cost.backward()
            optimizer.step()
            optimizer.zero_grad()
        print('Epoch: {:03d}, Loss: {:.5f}'.format(epoch, cost.item()))

    model.eval()
    y_pred = []
    y_true = []
    for time, snapshot in enumerate(test_dataset):
        snapshot.to(device)
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
        y_pred.append(y_hat.detach().cpu().numpy())
        y_true.append(snapshot.y.detach().cpu().numpy())
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    print('Test Accuracy: {:.4f}'.format((y_pred.round() == y_true).mean()))

    del model
    torch.cuda.empty_cache()

test_1layer_GConvGRU(data_df)

cpu
Epoch: 000, Loss: 0.36342


KeyboardInterrupt: 

GConvLSTM

In [16]:
def test_1layer_GConvLSTM(df):
    x = np.zeros([df.shape[0], 3, 2])
    x[:, 0, 0] = df['Prcp1'].values
    x[:, 0, 1] = df['SM1'].values
    x[:, 1, 0] = df['Prcp2'].values
    x[:, 1, 1] = df['SM2'].values
    x[:, 2, 0] = df['Prcp3'].values
    x[:, 2, 1] = df['SM3'].values

    y = np.zeros([df.shape[0], 3, 1])
    y[:, 0, 0] = df['flood1'].values
    y[:, 1, 0] = df['flood2'].values
    y[:, 2, 0] = df['flood3'].values

    dataset = StaticGraphTemporalSignal(
        edge_index = np.array([[0, 2], [1, 1]], dtype=np.compat.long),
        edge_weight = np.array([1.0, 1.0], dtype=float),
        features = x,
        targets = y
    )

    train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.8)

    class TemporalGNN(torch.nn.Module):
        def __init__(self, dim_in):
            super().__init__()
            self.recurrent = GConvLSTM(dim_in, 16, 1)
            self.linear = Linear(16, 1)

        def forward(self, x, edge_index, edge_weight):
            x, _ = self.recurrent(x, edge_index, edge_weight)
            x = x.relu()
            x = self.linear(x)
            return x.sigmoid()
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(device)
    model = TemporalGNN(dim_in=2).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.BCELoss().to(device)

    model.train()
    for epoch in range(10):
        cost = 0
        for time, snapshot in enumerate(train_dataset):
            snapshot.to(device)
            y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
            cost = criterion(y_hat, snapshot.y)
            cost.backward()
            optimizer.step()
            optimizer.zero_grad()
        print('Epoch: {:03d}, Loss: {:.5f}'.format(epoch, cost.item()))

    model.eval()
    y_pred = []
    y_true = []
    for time, snapshot in enumerate(test_dataset):
        snapshot.to(device)
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
        y_pred.append(y_hat.detach().cpu().numpy())
        y_true.append(snapshot.y.detach().cpu().numpy())
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    print('Test Accuracy: {:.4f}'.format((y_pred.round() == y_true).mean()))

    del model
    torch.cuda.empty_cache()

test_1layer_GConvLSTM(data_df)

cpu
Epoch: 000, Loss: 0.39815
Epoch: 001, Loss: 0.17005
Epoch: 002, Loss: 0.11025
Epoch: 003, Loss: 0.09311
Epoch: 004, Loss: 0.08660
Epoch: 005, Loss: 0.08353
Epoch: 006, Loss: 0.08192
Epoch: 007, Loss: 0.08101
Epoch: 008, Loss: 0.08057
Epoch: 009, Loss: 0.08030
Test Accuracy: 0.9178


In [17]:
def test_1layer_GConvLSTM(df):
    x = np.zeros([df.shape[0], 3, 2])
    x[:, 0, 0] = df['Prcp1'].values
    x[:, 0, 1] = df['SM1'].values
    x[:, 1, 0] = df['Prcp2'].values
    x[:, 1, 1] = df['SM2'].values
    x[:, 2, 0] = df['Prcp3'].values
    x[:, 2, 1] = df['SM3'].values

    y = np.zeros([df.shape[0], 3, 1])
    y[:, 0, 0] = df['flood1'].values
    y[:, 1, 0] = df['flood2'].values
    y[:, 2, 0] = df['flood3'].values

    dataset = StaticGraphTemporalSignal(
        edge_index = np.array([[0, 2], [1, 1]], dtype=np.compat.long),
        edge_weight = np.array([1.0, 1.0], dtype=float),
        features = x,
        targets = y
    )

    train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.8)

    class TemporalGNN(torch.nn.Module):
        def __init__(self, dim_in):
            super().__init__()
            self.recurrent = GConvLSTM(dim_in, 16, 1)
            self.linear = Linear(16, 1)

        def forward(self, x, edge_index, edge_weight):
            x, _ = self.recurrent(x, edge_index, edge_weight)
            x = x.relu()
            x = self.linear(x)
            return x.sigmoid()
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(device)
    model = TemporalGNN(dim_in=2).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.BCELoss().to(device)

    model.train()
    for epoch in range(20):
        cost = 0
        for time, snapshot in enumerate(train_dataset):
            snapshot.to(device)
            y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
            cost = criterion(y_hat, snapshot.y)
            cost.backward()
            optimizer.step()
            optimizer.zero_grad()
        print('Epoch: {:03d}, Loss: {:.5f}'.format(epoch, cost.item()))

    model.eval()
    y_pred = []
    y_true = []
    for time, snapshot in enumerate(test_dataset):
        snapshot.to(device)
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
        y_pred.append(y_hat.detach().cpu().numpy())
        y_true.append(snapshot.y.detach().cpu().numpy())
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    print('Test Accuracy: {:.4f}'.format((y_pred.round() == y_true).mean()))

    del model
    torch.cuda.empty_cache()

test_1layer_GConvLSTM(data_df)

cpu
Epoch: 000, Loss: 0.21905
Epoch: 001, Loss: 0.11648
Epoch: 002, Loss: 0.08965
Epoch: 003, Loss: 0.09675
Epoch: 004, Loss: 0.10131
Epoch: 005, Loss: 0.09003
Epoch: 006, Loss: 0.09843
Epoch: 007, Loss: 0.10392
Epoch: 008, Loss: 0.10088
Epoch: 009, Loss: 0.10736
Epoch: 010, Loss: 0.11733
Epoch: 011, Loss: 0.12307
Epoch: 012, Loss: 0.13162
Epoch: 013, Loss: 0.11832
Epoch: 014, Loss: 0.11951
Epoch: 015, Loss: 0.11382
Epoch: 016, Loss: 0.12555
Epoch: 017, Loss: 0.12050
Epoch: 018, Loss: 0.09819
Epoch: 019, Loss: 0.12323
Test Accuracy: 0.9315


In [18]:
def test_1layer_GConvLSTM(df):
    x = np.zeros([df.shape[0], 3, 2])
    x[:, 0, 0] = df['Prcp1'].values
    x[:, 0, 1] = df['SM1'].values
    x[:, 1, 0] = df['Prcp2'].values
    x[:, 1, 1] = df['SM2'].values
    x[:, 2, 0] = df['Prcp3'].values
    x[:, 2, 1] = df['SM3'].values

    y = np.zeros([df.shape[0], 3, 1])
    y[:, 0, 0] = df['flood1'].values
    y[:, 1, 0] = df['flood2'].values
    y[:, 2, 0] = df['flood3'].values

    dataset = StaticGraphTemporalSignal(
        edge_index = np.array([[0, 2], [1, 1]], dtype=np.compat.long),
        edge_weight = np.array([1.0, 1.0], dtype=float),
        features = x,
        targets = y
    )

    train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.8)

    class TemporalGNN(torch.nn.Module):
        def __init__(self, dim_in):
            super().__init__()
            self.recurrent = GConvLSTM(dim_in, 16, 1)
            self.linear = Linear(16, 1)

        def forward(self, x, edge_index, edge_weight):
            x, _ = self.recurrent(x, edge_index, edge_weight)
            x = x.relu()
            x = self.linear(x)
            return x.sigmoid()
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(device)
    model = TemporalGNN(dim_in=2).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.BCELoss().to(device)

    model.train()
    for epoch in range(50):
        cost = 0
        for time, snapshot in enumerate(train_dataset):
            snapshot.to(device)
            y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
            cost = criterion(y_hat, snapshot.y)
            cost.backward()
            optimizer.step()
            optimizer.zero_grad()
        print('Epoch: {:03d}, Loss: {:.5f}'.format(epoch, cost.item()))

    model.eval()
    y_pred = []
    y_true = []
    for time, snapshot in enumerate(test_dataset):
        snapshot.to(device)
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
        y_pred.append(y_hat.detach().cpu().numpy())
        y_true.append(snapshot.y.detach().cpu().numpy())
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    print('Test Accuracy: {:.4f}'.format((y_pred.round() == y_true).mean()))

    del model
    torch.cuda.empty_cache()

test_1layer_GConvLSTM(data_df)

cpu
Epoch: 000, Loss: 0.16664
Epoch: 001, Loss: 0.09381
Epoch: 002, Loss: 0.09427
Epoch: 003, Loss: 0.09333
Epoch: 004, Loss: 0.08614
Epoch: 005, Loss: 0.09777
Epoch: 006, Loss: 0.09578
Epoch: 007, Loss: 0.10823
Epoch: 008, Loss: 0.11386
Epoch: 009, Loss: 0.12289
Epoch: 010, Loss: 0.11846
Epoch: 011, Loss: 0.12164
Epoch: 012, Loss: 0.12467
Epoch: 013, Loss: 0.12719
Epoch: 014, Loss: 0.12878
Epoch: 015, Loss: 0.13060
Epoch: 016, Loss: 0.12836
Epoch: 017, Loss: 0.14596
Epoch: 018, Loss: 0.14092
Epoch: 019, Loss: 0.13220
Epoch: 020, Loss: 0.14047
Epoch: 021, Loss: 0.14707
Epoch: 022, Loss: 0.08691
Epoch: 023, Loss: 0.14055
Epoch: 024, Loss: 0.14856
Epoch: 025, Loss: 0.15062
Epoch: 026, Loss: 0.14166
Epoch: 027, Loss: 0.14305
Epoch: 028, Loss: 0.15228
Epoch: 029, Loss: 0.14792
Epoch: 030, Loss: 0.14359
Epoch: 031, Loss: 0.13324
Epoch: 032, Loss: 0.13283
Epoch: 033, Loss: 0.15027
Epoch: 034, Loss: 0.15526
Epoch: 035, Loss: 0.14832
Epoch: 036, Loss: 0.15670
Epoch: 037, Loss: 0.15806
Epoch: 0

In [19]:
def test_1layer_GConvLSTM(df):
    x = np.zeros([df.shape[0], 3, 2])
    x[:, 0, 0] = df['Prcp1'].values
    x[:, 0, 1] = df['SM1'].values
    x[:, 1, 0] = df['Prcp2'].values
    x[:, 1, 1] = df['SM2'].values
    x[:, 2, 0] = df['Prcp3'].values
    x[:, 2, 1] = df['SM3'].values

    y = np.zeros([df.shape[0], 3, 1])
    y[:, 0, 0] = df['flood1'].values
    y[:, 1, 0] = df['flood2'].values
    y[:, 2, 0] = df['flood3'].values

    dataset = StaticGraphTemporalSignal(
        edge_index = np.array([[0, 2], [1, 1]], dtype=np.compat.long),
        edge_weight = np.array([1.0, 1.0], dtype=float),
        features = x,
        targets = y
    )

    train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.8)

    class TemporalGNN(torch.nn.Module):
        def __init__(self, dim_in):
            super().__init__()
            self.recurrent = GConvLSTM(dim_in, 16, 1)
            self.linear = Linear(16, 1)

        def forward(self, x, edge_index, edge_weight):
            x, _ = self.recurrent(x, edge_index, edge_weight)
            x = x.relu()
            x = self.linear(x)
            return x.sigmoid()
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(device)
    model = TemporalGNN(dim_in=2).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.BCELoss().to(device)

    model.train()
    for epoch in range(100):
        cost = 0
        for time, snapshot in enumerate(train_dataset):
            snapshot.to(device)
            y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
            cost = criterion(y_hat, snapshot.y)
            cost.backward()
            optimizer.step()
            optimizer.zero_grad()
        print('Epoch: {:03d}, Loss: {:.5f}'.format(epoch, cost.item()))

    model.eval()
    y_pred = []
    y_true = []
    for time, snapshot in enumerate(test_dataset):
        snapshot.to(device)
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
        y_pred.append(y_hat.detach().cpu().numpy())
        y_true.append(snapshot.y.detach().cpu().numpy())
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    print('Test Accuracy: {:.4f}'.format((y_pred.round() == y_true).mean()))

    del model
    torch.cuda.empty_cache()

test_1layer_GConvLSTM(data_df)

cpu
Epoch: 000, Loss: 0.21687
Epoch: 001, Loss: 0.11354
Epoch: 002, Loss: 0.10237
Epoch: 003, Loss: 0.09295
Epoch: 004, Loss: 0.09802
Epoch: 005, Loss: 0.09756
Epoch: 006, Loss: 0.09908
Epoch: 007, Loss: 0.10209
Epoch: 008, Loss: 0.10134
Epoch: 009, Loss: 0.10405
Epoch: 010, Loss: 0.10539
Epoch: 011, Loss: 0.11359
Epoch: 012, Loss: 0.10897
Epoch: 013, Loss: 0.10230
Epoch: 014, Loss: 0.05497
Epoch: 015, Loss: 0.11593
Epoch: 016, Loss: 0.11319
Epoch: 017, Loss: 0.11324
Epoch: 018, Loss: 0.11110
Epoch: 019, Loss: 0.13328
Epoch: 020, Loss: 0.13784
Epoch: 021, Loss: 0.13428
Epoch: 022, Loss: 0.14441
Epoch: 023, Loss: 0.13970
Epoch: 024, Loss: 0.14839
Epoch: 025, Loss: 0.14961
Epoch: 026, Loss: 0.15121
Epoch: 027, Loss: 0.15225
Epoch: 028, Loss: 0.15290
Epoch: 029, Loss: 0.15374
Epoch: 030, Loss: 0.15217
Epoch: 031, Loss: 0.14188
Epoch: 032, Loss: 0.15376
Epoch: 033, Loss: 0.15477
Epoch: 034, Loss: 0.15605
Epoch: 035, Loss: 0.15709
Epoch: 036, Loss: 0.15543
Epoch: 037, Loss: 0.15482
Epoch: 0